In [1]:
import csv

class Item:
    def __init__(self, lvl : int, prof : str, atype : str, name : str, stats : dict):
        self.id = f'{name}-{atype}-{lvl}-{prof}'
        self.lvl = lvl
        self.prof = prof
        self.atype = atype
        self.name = name
        self.stats = stats
        
    def toList(self):
        return [self.lvl, self.prof, self.atype, self.name, self.stats]
        
class Eq(Item):
    def __init__(self):
        self.items = []
        self.stats = {}
        
    def check(self, item : Item):
        # Check compatibility of classes with 1st item in eq
        zipped = set(zip(self.items[0].prof, item.prof))
        for el in zipped:
            if el[0] == el[1]:
                return True
        return False
        
    def add(self, item : Item):
        # Check if new item type existed before
        self.remove(item.atype)
        # Append new item to eq
        self.items.append(item)
        for key, val in item.stats.items():
            if key in self.stats:
                self.stats[key] += val
            else:
                self.stats[key] = val
                
    def remove(self, acctype : str):
        for i in self.items:
            if i.atype == acctype:
                self.items.remove(i)
                for key, val in i.stats.items():
                    if key in self.stats:
                        self.stats[key] -= val
                    if self.stats[key] == 0:
                        del self.stats[key]

# write to file serialized items
def csvSerializer(items : list, item_type : str, output_file : str):
    with open(output_file, "w", encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        for el in items:
            # Split each item's data
            split_lst = el['data-stat'].split(';')
            # return & split by only those which contain '='
            params = [p.split('=') for p in split_lst if "=" in p]
            dct = {p[0] : p[1] for p in params}
            try:
                single_item = Item(int(dct['lvl']),
                     dct['reqp'], 
                     item_type, 
                     el['data-name'], 
                     {stats[p] : dct[p] for p in dct if p in stats})
                #print(single_item.toList())
                writer.writerow(single_item.toList())
            except:
                pass

In [2]:
# Dictionary of possible statistics derived from game items translated into new names

stats = {
'da':'all_features',
'ds':'strength',
'dz':'agility',
'di':'intellect',
'crit':'crit_strike',
'critval':'physical_crit',
'critmval':'magical_crit',
'energybon':'energy',
'manabon':'mana',
'endest':'energy_subtra',
'manadest':'mana_subtra',
'resdmg':'resist_reduction',
'acdmg':'armor_reduction',
'abdest':'absorb_reduction',
'hp':'hp',
'heal':'hp_regen',
'sa':'attack_speed',
'slow':'as_reduction',
'evade':'dodge',
'lowevade':'dodge_reduction',
'lowcrit':'crit_reduction',
'dmg':'physical_dmg',
'fire':'fire_dmg',
'light':'light_dmg',
'frost':'frost_dmg',
'frost_slow':'frost_slow',
'act':'poison_resist',
'resfire':'fire_resist',
'reslight':'light_resist',
'resfrost':'frost_resist',
'pierce':'armor_pierce',
'lowcritallval':'crit_power_reduction',
'lowheal2turns':'hp_regen_reduction',
'resacdmg':'armor_reduction_protection',
'resmanaendest':'mana_subtra_protection',
'ac':'armor',
'absorb':'physical_absorb',
'absorbm':'magical_absorb',
'blok':'block',
'hpbon':'strength_hp',
'adest':'hp_regen_reduction',
'contra':'contra',
'pierceb':'pierce_block',
'poison':'poison_dmg',
'poison_slow':'poison_slow',
'wound':'wound_dmg',
'legbon':'legendary_bonus'
}

# Items' types for all 6 game professions (classes):

weapon_types = {
    'blade_dancer' : ['1h', 'sb', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl'],
    'warrior' : ['1h', '2h', 'hh', 'sh', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl'],
    'paladin' : ['1h', 'hh', 'sh', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl'],
    'mage' : ['wa', 'mo', 'sh', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl'],
    'tracker' : ['rw', 'am', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl'],
    'hunter' : ['rw', 'am', 'ar', 'he', 'bo', 'gl', 'ri', 'ne', 'bl']
}

# Id's of accessory types at margohelp url for each proffesion:

prof_translator = {
    'blade_dancer' : (1, 5, 8, 9, 10, 11, 12, 13),
    'warrior' : (1, 2, 3, 14, 8, 9, 10, 11, 12, 13),
    'paladin' : (1, 3, 14, 8, 9, 10, 11, 12, 13),
    'mage' : (6, 7, 14, 8, 9, 10, 11, 12, 13),
    'tracker': (4, 21, 8, 9, 10, 11, 12, 13),
    'hunter': (4, 21, 8, 9, 10, 11, 12, 13)
}

# page url items' types translator:

item_translator = {
    1 : 'Jednoręczne', # One-handed
    2 : 'Dwuręczne', # Two-handed
    3 : 'Półtoraręczne', # Hand&half
    4 : 'Dystansowe', # Ranged
    5 : 'Pomocnicza', # Secondary
    6 : 'Różdżki', # Rods
    7 : 'Orbymagiczne', # Magical orbs
    8 : 'Zbroje', # Armor
    9 : 'Hełmy', # Headgear
    10 : 'Buty', # Boots
    11 : 'Rękawice', # Gloves
    12 : 'Pierścienie', # Rings
    13 : 'Naszyjniki', # Necklaces
    14 : 'Tarcze', # Shields
    21 : 'Strzały' # Arrows
}

proffesions = {
    'w':'Wojownik', # Warrior
    'b':'Tancerzostrzy', # Blade dancer
    'p':'Paladyn', # Paladin
    'm':'Mag', # Mage
    't':'Tropiciel', # Tracker
    'h':'Łowca' # Hunter
}

In [4]:
import requests
from bs4 import BeautifulSoup
from time import sleep

# Blade dancer url:
#https://margohelp.pl/przedmioty/b/1/4/TancerzOstrzy/Jednor%C4%99czne/Heroiczne/1-150/
# Paladin url:
#https://margohelp.pl/przedmioty/p/1/4/Paladyn/Jednor%C4%99czne/Heroiczne/1-150/
# Warrior url:
#https://margohelp.pl/przedmioty/w/1/4/Wojownik/Jednor%C4%99czne/Heroiczne/1-150/
# Mage url:
#https://margohelp.pl/przedmioty/m/6/4/Mag/R%C3%B3%C5%BCd%C5%BCki/Heroiczne/1-150/
# Tracker url:
#https://margohelp.pl/przedmioty/t/4/4/Tropiciel/Dystansowe/Heroiczne/1-150/
# Hunter url:
#https://margohelp.pl/przedmioty/h/4/4/%C5%81owca/Dystansowe/Heroiczne/1-150/

# Function for downloading each proffesion with their item type items in separate .csv file
def scrapeAndSaveData():
    for key, vals in prof_translator.items():
        for idx, acc in enumerate(vals):
            page_items = []
            for i in (0, 150):
                sleep(1)
                page = requests.get(f"https://margohelp.pl/przedmioty/{key[0]}/{acc}/4/{proffesions[key[0]]}/{item_translator[acc]}/Heroiczne/{1+i}-{150+i}/")
                soup = BeautifulSoup(page.content, 'html.parser')
                page_items.extend(soup.find_all(class_='item'))
            csvSerializer(page_items, weapon_types[key][idx], f'itemy/{proffesions[key[0]]}_{item_translator[acc]}.csv')

In [ ]:
import numpy as np
import secrets

class Player:
    def __init__(self, lvl : int, prof : str, eq_stats : dict):
        self.id = secrets.token_hex(16)
        self.proffesion = prof
        self.eq_stats = eq_stats

def fightSimulator(p1, p2):
    
    initial_stats = []
    updatibles = []
    final_stats = []
        
        